In [107]:
import itertools
import operator
from functools import reduce
from itertools import combinations
import numpy as np

## Mã hóa
Với mã Reed-Muller, ta mã hóa đơn giản bằng cách nhân với ma trận sinh G.<br/>
Mỗi hàng của ma trận là 1 đơn thức với số biến tăng dần. Ma trận sinh $G$ được tạo như sau: 
- Hàm $0$ được biểu diễn bởi vector gồm $2^m$ số 1.
- Tạo các đơn thức gồm 1 biến bằng cách lặp lại giá trị $0$ và $1$. Hàm $x_i$ được sinh bằng cách nối $2^i$ số $0$ và $2^i$ số $1$, lặp lại đến khi được vector độ dài $2^m$
- Nhân các hàm 1 biến $x_i$ để được các đơn thức bậc cao

## Giải mã
Với cách mã hóa bằng việc nhân ma trận như trên, ta có thể giải mã bằng thuật toán Reed.<br/>
Ta định nghĩa i-flat là một coset $a + K$ với $K$ là một không gian con $i$ chiều của $Z^m_2$. Mỗi i-flat được biểu diễn qua một vector đặc trưng có độ dài $2^m$, với giá trị ở vị trí $k$ bằng 1 nếu $i$ thuộc i-flat đó.<br/>
Để giải mã, ta tìm tính chẵn lẻ của 0-flat. Việc này được thực hiện một cách đệ quy như sau:
- Tính chẵn lẻ của i-flat được tính qua biểu quyết số đông. Nếu trong số các i+1-flat chứa nó, số flat lẻ nhiều hơn thì i-flat đó được tính là lẻ, ngược lại nó được xem là chẵn.
- Tính chẵn lẻ của r+1-flat được tính bằng cách lấy tích trong mô đun 2 với vector cần giải mã.

Để sinh ra tất cả các flat, ta có tính chất sau:
- Mỗi không gian con k chiều $K$ có thể mở rộng thành không gian con k+1 chiều $K'$ bằng cách thêm một phần tử không thuộc $K$ vào tập cơ sở.

Các 0-flat là các tập hợp gồm 1 phần tử. Ta tạo i+1-flat $F'$ từ i-flat $F$ bằng cách chọn một giá trị $b$ nhỏ hơn $2^m$, cộng mỗi phần tử trong $F$ với b (phép cộng trên $Z^m_2$ được thực hiện bằng phép XOR) và thêm vào $F'$, rồi hợp với $F$. Nếu $|F'| = 2|F|$ thì $F'$ là một i+1-flat.

In [108]:
class ReedMuller:
    def __init__(self, m, r):
        self.m = m
        self.r = r
        self.n = 2**m
        self.G = self.__generating_matrix()
        self.flat_list=self.__generate_flats()
        
    def __generate_flats(self):
        """Construct a list of flats"""
        flat_list = [[{i} for i in range(self.n)]]
        for i in range(1, self.r + 2):
            i_flat = []
            for sub_flat in flat_list[i - 1]:
                for e in range(1, self.n):
                    flat = sub_flat.union({e ^ j for j in sub_flat})
                    if len(flat) == 2**i and flat not in i_flat:
                        i_flat.append(flat)
            flat_list.append(list(i_flat))
        return flat_list
            
    def __variable_func(self, var):
        """Create a list of n variables monomials"""
        return sum([[0] * 2**var + [1] * 2**var for i in range(2**(self.m - var - 1))], [])
    
    def __logical_product(self, f, g):
        return [i * j for (i, j) in zip(f, g)]
    
    def __generating_matrix(self):
        """Construct generator matrix"""
        variable_funcs = [self.__variable_func(i) for i in range(self.m)]
        monomial_list = [reduce(self.__logical_product, [variable_funcs[i] for i in index_list]) for num_var in range(2, self.r + 1) for index_list in combinations(range(self.m), num_var)]
        matrix = [[1] * 2**self.m] + variable_funcs + monomial_list
        return np.array(matrix)
    
    def encoding(self, data):
        assert len(data) == self.G.shape[0]
        return data.dot(self.G) % 2
    
    def __characteristic_func(self, flat):
        """Construct characteristic function of flat from elements"""
        vector = np.zeros(self.n)
        vector[list(flat)] = 1
        return vector
    
    def decoding(self, word):
        word = np.array(word)
        parity = [[] for i in range(self.r + 2)]
        for i in range(self.r + 1, -1, -1):
            if i == self.r + 1:
                for flat in self.flat_list[i]:
                    parity[i].append(word.dot(self.__characteristic_func(flat)) % 2)
            else:
                for flat in self.flat_list[i]:
                    count = []
                    for index in range(len(parity[i + 1])):
                        if flat.issubset(self.flat_list[i + 1][index]):
                            count.append(parity[i + 1][index])
                    if sum(count) * 2 > len(count):
                        parity[i].append(1)
                    else:
                        parity[i].append(0)
        e = []
        for i, p in enumerate(parity[0]):
            if p:
                word[i] ^= 1
                e.append(i)
        return word, e

In [109]:
data = np.random.randint(low = 0, high = 2, size = (4, ))
data

array([0, 1, 0, 1])

In [149]:
import random
def noise(word, num_e):
    data = word.copy()
    corrupted_pos = np.random.randint(0, high = len(data) - 1, size = num_e)
    data[corrupted_pos] ^= 1
    return data, corrupted_pos

In [150]:
rm = ReedMuller(3, 1)
word = rm.encoding(data)
word

array([0, 1, 0, 1, 1, 0, 1, 0], dtype=int32)

In [151]:
corrupted, e = noise(word, 2)
corrupted, e

(array([0, 1, 0, 0, 0, 0, 1, 0], dtype=int32), array([3, 4]))

In [106]:
correct = rm.decoding(corrupted)
correct

(array([1, 0, 0, 1, 1, 0, 0, 1], dtype=int32), [7])